# Importing

In [1]:
# !pip install tensorflow -q

In [2]:
# Import the necessary libraries
from datasets import load_dataset, load_metric
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention, Concatenate, Dropout
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import unicodedata
import pandas as pd
import numpy as np

2024-04-28 20:42:36.415169: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 20:42:36.415255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 20:42:36.538721: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data Preprocessing

In [3]:
# Load the Data
datasets = load_dataset("flax-sentence-embeddings/stackexchange_title_best_voted_answer_jsonl", "scifi")
datasets

Generating train split:   0%|          | 0/54805 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title_body', 'upvoted_answer'],
        num_rows: 54805
    })
})

In [4]:
dataset = datasets["train"]
dataset

Dataset({
    features: ['title_body', 'upvoted_answer'],
    num_rows: 54805
})

In [5]:
df = dataset.to_pandas()
df = df.rename(columns={'title_body': 'question', 'upvoted_answer': 'answer'})
df

,question,answer
0,What H.P. Lovecraft stories do Charles Stross'...,The Cthulu Mythos of Lovecraft. \nThe Laundry ...
1,Why did Alexander Randall want Mary Hawkins to...,I think the simple answer is that it would be ...
2,Cyborg Movie from 1980s or 1990s,"Could this be the big-budget blockbuster ""Elim..."
3,Would Voldemort (or Thicknesse) have bothered ...,Is it likely that Voldemort or Thicknesse chan...
4,"Is ""Room of lies"" a Star Wars reference?","Although it's hard to prove a negative, there ..."
...,...,...
54800,Why aren't the Valar considered gods?,They are.\n\nThat's a bit of a simplistic and ...
54801,"In Gunnerkrigg Court, what happened to the see...",I don't think any of your questions have been ...
54802,Noir detective story. Comet hits city and phys...,It might be M. John Harrison's 2006 novel Nova...
54803,Why did Wade and his team enter into the Shini...,It was not so much that Wade and company were ...


In [6]:
df = df[:10000]
df

,question,answer
0,What H.P. Lovecraft stories do Charles Stross'...,The Cthulu Mythos of Lovecraft. \nThe Laundry ...
1,Why did Alexander Randall want Mary Hawkins to...,I think the simple answer is that it would be ...
2,Cyborg Movie from 1980s or 1990s,"Could this be the big-budget blockbuster ""Elim..."
3,Would Voldemort (or Thicknesse) have bothered ...,Is it likely that Voldemort or Thicknesse chan...
4,"Is ""Room of lies"" a Star Wars reference?","Although it's hard to prove a negative, there ..."
...,...,...
9995,Why didn't Sauron fight in the final battle?,"Book 5, Chapter 4:\n\n\n 'Is Faramir come?' [..."
9996,Orphan siblings from another planet search the...,"I knew I'd get there originally, although it t..."
9997,How does the Dust of Broken Hearts choose its ...,The implication from the film's official novel...
9998,Why did the Weasleys decide to have so many ch...,"In Ron's opinion, it was because their mother ..."


In [7]:
stop_words = set(stopwords.words('english'))
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [8]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

# Function for preprocessing text
def preprocess_text(text):
    # Convert text to lowercase
    text = unicode_to_ascii(text.lower().strip())
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)
    text = ' '.join([contractions[word] if word in contractions else word for word in text.split()])
    text =  "<sos> " +  text + "<eos>"
    return text

# Apply preprocessing to question and answer columns
preprocessed_df = df.copy()
preprocessed_df['question'] = preprocessed_df['question'].apply(preprocess_text)
preprocessed_df['answer'] = preprocessed_df['answer'].apply(preprocess_text)

# Print the preprocessed DataFrame
preprocessed_df

,question,answer
0,<sos> what h p lovecraft stories do charles st...,<sos> the cthulu mythos of lovecraft the laund...
1,<sos> why did alexander randall want mary hawk...,<sos> i think the simple answer is that it wou...
2,<sos> cyborg movie from or<eos>,<sos> could this be the big budget blockbuster...
3,<sos> would voldemort or thicknesse have bothe...,<sos> is it likely that voldemort or thickness...
4,<sos> is room of lies a star wars reference<eos>,<sos> although it s hard to prove a negative t...
...,...,...
9995,<sos> why didn t sauron fight in the final bat...,<sos> book chapter is faramir come denethor as...
9996,<sos> orphan siblings from another planet sear...,<sos> i knew i d get there originally although...
9997,<sos> how does the dust of broken hearts choos...,<sos> the implication from the film s official...
9998,<sos> why did the weasleys decide to have so m...,<sos> in ron s opinion it was because their mo...


In [9]:
# Preprocessing the data
questions = preprocessed_df['question'].values.tolist()
answers = preprocessed_df['answer'].values.tolist()

# Tokenizing the data
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(np.concatenate((questions, answers), axis=0))

vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
question_seqs = tokenizer.texts_to_sequences(questions)
answer_seqs = tokenizer.texts_to_sequences(answers)

# Padding sequences for equal length
# Pad sequences for equal length
max_len_question = max(len(seq) for seq in question_seqs)
max_len_answer = max(len(seq) for seq in answer_seqs)
max_len = max(max_len_question, max_len_answer)
max_len = 60
print(max_len)
# Pad sequences separately for questions and answers
question_seqs = pad_sequences(question_seqs, maxlen=max_len, padding='post')
answer_seqs = pad_sequences(answer_seqs, maxlen=max_len, padding='post')

60


In [10]:
tokenizer.texts_to_sequences("<sos>")

[[], [10], [839], [10], []]

In [11]:
tokenizer.word_index["<sos>"]

13

# Defining the Model

In [12]:
# Define the model architecture
latent_dim = 256  # Dimensionality of the encoding space

# Encoder
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(vocab_size, latent_dim, input_shape=(max_len,))
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.1)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding(encoder_inputs))
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len-1,))
decoder_embedding = Embedding(vocab_size, latent_dim, input_shape=(max_len-1,))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.1, recurrent_dropout=0.1)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding(decoder_inputs), initial_state=encoder_states)

# Attention mechanism
attention_layer = Attention()
attention_output = attention_layer([decoder_outputs, encoder_outputs])

# Concatenate attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Add dropout layer for regularization
decoder_concat_input = Dropout(0.1)(decoder_concat_input)

# Output layer
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:89: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 59)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 60, 256)   │ 16,661,504 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 59, 256)   │ 16,661,504 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 60, 256), │    525,312 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 59, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 59, 256)   │          0 │ lstm_1[0][0],     │
│ (Attention)         │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 59, 512)   │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 59, 512)   │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 59, 65084) │ 33,388,092 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 67,761,724 (258.49 MB)

 Trainable params: 67,761,724 (258.49 MB)

 Non-trainable params: 0 (0.00 B)

# Training and Saving the Model

In [14]:
# Train the model
model.fit([question_seqs, answer_seqs[:, :-1]], answer_seqs[:, 1:],
          batch_size=32,
          epochs=150,
          validation_split=0.2)

Epoch 1/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 61s 219ms/step - loss: 7.8564 - val_loss: 6.7457
Epoch 2/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 6.4572 - val_loss: 6.4452
Epoch 3/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 6.0889 - val_loss: 6.3227
Epoch 4/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 5.8382 - val_loss: 6.2630
Epoch 5/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 5.6297 - val_loss: 6.2332
Epoch 6/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 5.4282 - val_loss: 6.2309
Epoch 7/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 5.2350 - val_loss: 6.2293
Epoch 8/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 215ms/step - loss: 5.0719 - val_loss: 6.2480
Epoch 9/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 216ms/step - loss: 4.8985 - val_loss: 6.2785
Epoch 10/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 4.7202 - val_loss: 6.3162
Epoch 11/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 4.5381 - val_loss: 6.3607
Epoch 12

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.6165 - val_loss: 10.9151
Epoch 138/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.6166 - val_loss: 10.9502
Epoch 139/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 217ms/step - loss: 0.6129 - val_loss: 10.9674
Epoch 140/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.6005 - val_loss: 10.9848
Epoch 141/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.6059 - val_loss: 11.0041
Epoch 142/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.6033 - val_loss: 11.0246
Epoch 143/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 0.5940 - val_loss: 11.0575
Epoch 144/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 0.5908 - val_loss: 11.0725
Epoch 145/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 217ms/step - loss: 0.5859 - val_loss: 11.0797
Epoch 146/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5884 - val_loss: 11.0937
Epoch 147/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5808 - val_loss:

In [19]:
# Save the model
model.save('path_to_save_model.h5')

In [20]:
xy=load_model('path_to_save_model.h5')

In [21]:
xy

<Functional name=functional_1, built=True>

In [17]:
# Function to generate a response given a input sentence
def generate_response(input_text):
    # Tokenize the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    # Pad the input sequence
    input_sequence = pad_sequences(input_sequence, maxlen=max_len, padding='post')
    
    # Initialize the decoder input sequence with start token
    decoder_input_sequence = np.zeros((1, max_len-1))
    decoder_input_sequence[0, 0] = tokenizer.word_index['<sos>']
    
    # Generate response using the trained model
    for i in range(max_len - 2):
        predictions = xy.predict([input_sequence, decoder_input_sequence])
        predicted_id = np.argmax(predictions[0, i, :])
        if 'eos' in tokenizer.word_index and predicted_id == tokenizer.word_index['<eos>']:
            break
        decoder_input_sequence[0, i+1] = predicted_id
    
    # Convert output sequence to text
    output_text = ''
    for token_index in decoder_input_sequence[0]:
        if 'eos' in tokenizer.word_index and token_index == tokenizer.word_index['<eos>'] or token_index == 0:
            break
        output_text += tokenizer.index_word[token_index] + ' '
    
    return output_text.strip()

# Test the function with input "how are you"
input_text = "how do you do"
response = generate_response(input_text)
print("Response:", response[5:])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━

In [18]:
print(response)

<sos> from tales of the king the silmarillion akallabeth quenta silmarillion of eldamar and the princes of the eldalie of the sundering of which are generally called in strength in the middle of the fallen kingdom and then we are needed and could have seen the silmarillion of which the sundering of which is ultimately stated explicitly is listed


In [26]:
# Function to generate a response given a input sentence
def generate_response(input_text):
    # Tokenize the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    # Pad the input sequence
    input_sequence = pad_sequences(input_sequence, maxlen=max_len, padding='pre')
    
    # Initialize the decoder input sequence with start token
    decoder_input_sequence = np.zeros((1, max_len-1))
    decoder_input_sequence[0, 0] = tokenizer.word_index['<sos>']
    
    # Generate response using the trained model
    for i in range(max_len - 2):
        predictions = xy.predict([input_sequence, decoder_input_sequence])
        predicted_id = np.argmax(predictions[0, i, :])
        if 'eos' in tokenizer.word_index and predicted_id == tokenizer.word_index['<eos>']:
            break
        decoder_input_sequence[0, i+1] = predicted_id
    
    # Convert output sequence to text
    output_text = ''
    for token_index in decoder_input_sequence[0]:
        if 'eos' in tokenizer.word_index and token_index == tokenizer.word_index['<eos>'] or token_index == 0:
            break
        output_text += tokenizer.index_word[token_index] + ' '
    
    return output_text.strip()

# Test the function with input "how are you"
input_text = "can you tell me more about star wars?"
response = generate_response(input_text)
print("Response:", response[5:])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━

topics for nlp cb -> https://huggingface.co/datasets/flax-sentence-embeddings/stackexchange_title_best_voted_answer_jsonl
softwareengineering
scifi
anime - https://huggingface.co/datasets/CaptionEmporium/refined-anime-instruct-en-641k

chatbots:-

pretrained- 	https://www.kaggle.com/code/rajkumarl/conversational-ai-chatbot	

In [25]:
response

'<sos> it s not a good candidate for the upcoming books although i could also have been looking for it here is the way to me in the text s page that had it archived actually intended to me to me here s the good person the good story can be read on the good list of the original'

In [27]:
# Train the model
model.fit([question_seqs, answer_seqs[:, :-1]], answer_seqs[:, 1:],
          batch_size=32,
          epochs=150,
          validation_split=0.2)

Epoch 1/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5684 - val_loss: 11.2198
Epoch 2/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5633 - val_loss: 11.2055
Epoch 3/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5637 - val_loss: 11.2410
Epoch 4/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 0.5593 - val_loss: 11.2446
Epoch 5/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 0.5528 - val_loss: 11.2606
Epoch 6/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 0.5539 - val_loss: 11.2855
Epoch 7/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5452 - val_loss: 11.2923
Epoch 8/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5460 - val_loss: 11.3273
Epoch 9/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5433 - val_loss: 11.3562
Epoch 10/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - loss: 0.5373 - val_loss: 11.3673
Epoch 11/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 54s 216ms/step - loss: 0.5358 - val_loss: 11.36

In [28]:
# Save the model
model.save('model2.h5')

In [29]:
import_model = load_model('model2.h5')

In [37]:
# Function to generate a response given a input sentence
def generate_response(input_text):
    # Tokenize the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    # Pad the input sequence
    input_sequence = pad_sequences(input_sequence, maxlen=max_len, padding='pre')
    
    # Initialize the decoder input sequence with start token
    decoder_input_sequence = np.zeros((1, max_len-1))
    decoder_input_sequence[0, 0] = tokenizer.word_index['<sos>']
    
    # Generate response using the trained model
    for i in range(max_len - 2):
        predictions = import_model.predict([input_sequence, decoder_input_sequence])
        predicted_id = np.argmax(predictions[0, i, :])
        if 'eos' in tokenizer.word_index and predicted_id == tokenizer.word_index['<eos>']:
            break
        decoder_input_sequence[0, i+1] = predicted_id
    
    # Convert output sequence to text
    output_text = ''
    for token_index in decoder_input_sequence[0]:
        if 'eos' in tokenizer.word_index and token_index == tokenizer.word_index['<eos>'] or token_index == 0:
            break
        output_text += tokenizer.index_word[token_index] + ' '
    
    return output_text.strip()

# Test the function with input "how are you"
input_text = "Why did Batman design Robin's suit with only the underwear without pants?"
response = generate_response(input_text)
print("Response:", response[5:])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━

In [31]:
response

'<sos> we see the scene where he a bad object we see a site of light it seems likely that isn t so the same shot it seems to be a new and style i have a new shot of jedi it seems like a star but not it s likely the same thing that will be a new'